# 7 Object Tracking

## 7.1 Optical Flow

Optical Flow = Pattern of apparent motion of image objects between two consecutive frames caused by the movement of the camera.
Assumptions:
- Pixel intensities don't change between consecutive frames
- Neighboring pixels have similar motion

The method we use is the **Lucas-Kanade** method.
The method doesn't know whether the camera is moving or the scene/object is moving.

How it works:
- We select some points on a frame: we can define the region which contains a face that was detected with the techniques learned in section 6 or we efine a set of **sparse** (key-)points.
- The algorithm will try to follow/track those points on the next frame by detecting them using the assumptions specified above.

But what if we want to track **all** the points on a frame? Lucas-Kanade doesn't do that, but the **Gunner Farnebäck**'s algorithm does -- this is called **dense optical flow**: we look at the entire frame and highlight the points that are moving: returned image's pixel intensity means amount of motion: black, no motion; color depending on movement direction.

### 7.1.1 Lucas-Kanade: Sparse Optical Flow

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Lucas-Kanade works with sparse points
# We are going to use the Shi-Tomasi corners as sparse points to track
# We prepare the param dictionary for the corner detector here
# We can tune these params; note: max num of points selected 10
corner_track_params = dict(maxCorners=10,
                           qualityLevel=0.3,
                           minDistance=7,
                           blockSize=7)

In [5]:
# Parameters for the Lucas-Kanade method
# - window size: integration window size, trade-off between small & larger windows: small more senstive to smaller motions, bat misses large motoins and sometimes delivers noise
# - maxLevel: Lucas-Kanade uses image pyramids (0: org resolution, 1: 1/2 resolution, 2: 1/4)
# - criteria (to stop iterative method): number of iterations and error threshold to stop; note the order of criteria & values is different!
lk_params = dict(winSize=(200,200),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10, 0.03))

In [6]:
#cap = cv2.VideoCapture(0) # webcam input
cap = cv2.VideoCapture('../../data/hand_move.mp4') # recorded video input
# Grab first frame; each frame becomes the previous frame
ret, prev_frame = cap.read()
# The method works with mono images
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
# Points to track: Shi-Tomasi corners
prevPoints = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params) # ** passes a dictionary
# Visualization purposes
mask = np.zeros_like(prev_frame) # matching mask with 0s
ret = True
while ret:
    ret, frame = cap.read()
    if (ret):
        # gray
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # OPTICAL FLOW: Pyramid Lucas-Kanade
        # input: previous gray, current gray, previous points, params
        # output: status = 1 iff point has been found
        nextPoints, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPoints, None, **lk_params)
        # Select found points: previous and new
        good_new = nextPoints[status==1]
        good_prev = prevPoints[status==1]
        # Draw points and movement vectors previous -> new
        for i, (new,prev) in enumerate(zip(good_new,good_prev)):
            # ravel() = reshape(-1) = flatten
            x_new, y_new = new.ravel()
            x_prev, y_prev = prev.ravel()
            # Draw vector/line
            mask = cv2.line(mask, (x_new,y_new), (x_prev,y_prev), (0,255,0), 3)
            # Draw point circle
            frame = cv2.circle(frame, (x_new,y_new), 8, (0,0,255), -1)
        # We add the image and the mask (themask contains only lines and circles)
        img = cv2.add(frame,mask)
        cv2.imshow('tracking', img)
        # wait for ESC
        k = cv2.waitKey(30) & 0xFF
        if k == 27:
            break
        # Update: current values (frame and points) become previous
        prev_gray = frame_gray.copy()
        prevPoints = good_new.reshape(-1,1,2)
    else:
        break
# Clear
# Note: on my mac, this doesn't work
cv2.destroyAllWindows()
cap.release()

### 7.1.2 Dense Optical Flow: Farnabäck

We the entire image and color points that we believe are moving.
Basically, the same structure is used but with the function `cv2.calcOpticalFlowFarneback()`. Its params are:
* prev first 8-bit single-channel input image.
* next second input image of the same size and the same type as prev.
* flow computed flow image that has the same size as prev and type CV_32FC2.
* pyr_scale parameter, specifying the image scale (\<1) to build pyramids for each image
    * pyr_scale=0.5 means a classical pyramid, where each next layer is twice smaller than the previous one.
    
* levels number of pyramid layers including the initial image; levels=1 means that no extra layers are created and only the original images are used.
* winsize averaging window size
    * larger values increase the algorithm robustness to image
* noise and give more chances for fast motion detection, but yield more blurred motion field.
* iterations number of iterations the algorithm does at each pyramid level.
* poly_n size of the pixel neighborhood used to find polynomial expansion in each pixel
    * larger values mean that the image will be approximated with smoother surfaces, yielding more robust algorithm and more blurred motion field, typically poly_n =5 or 7.
* poly_sigma standard deviation of the Gaussian that is used to smooth derivatives used as a basis for the polynomial expansion; for poly_n=5, you can set poly_sigma=1.1, for poly_n=7, a good value would be poly_sigma=1.5.

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#cap = cv2.VideoCapture(0) # webcam input
cap = cv2.VideoCapture('../../data/hand_move.mp4') # recorded video input
ret, frame1 = cap.read()
# Get gray scale image of first frame and make a mask in HSV color
prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
# Mask for visualization - HSV colormap
hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255 # fully saturated; the other channels are going to be movement angle & magnitude!
ret = True
while ret:
    ret, frame2 = cap.read()
    if ret:
        nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
        # Check out the markdown text above for a break down of these paramters, most of these are just suggested defaults
        # output: flow is a 2-channel movement vector field with x (channel 0) & (1) y movement values
        # we transaform it later to polar coords to visualize it with HSV color mapping
        flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        # Color the channels based on the angle of travel
        # Pay close attention to your video, the path of the direction of flow will determine color!
        mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1], angleInDegrees=True)
        hsv_mask[:,:,0] = ang/2 # we scale to 180 deg in order to work with half the hues, it's easier to understand visualization
        hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX) # map magnitude values to [0,255]
        # Convert back to BGR to show with imshow from cv
        bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
        cv2.imshow('frame2',bgr)
        # Wait for ESC
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
        # Set the previous image as the next iamge for the loop
        prvsImg = nextImg
# Clear
cap.release()
cv2.destroyAllWindows()